## Load the Motion Data

Data is in x and y coordinators for each pixel. Each sample will be an array of 10 (frames) x 40 x 40 (capture window) x 2 (x and y) dimensions. 

In [ ]:
import numpy as np
import os
import matplotlib
matplotlib.use('TkAgg') # For displaying animation
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from helper_functions import *

In [ ]:
data_dir = 'sonic_pi_face/data/'

# Get list of data files
data_files = get_data_files(data_dir)

# Load data into a dictionary
# Note: Checks for incomplete data
data = get_gesture_data(data_files)

In [ ]:
# Get list of gestures in data
data.keys()

# Get number of `open-close` samples
len(data['open-close'])

# Load first data sample from `open-close`
data['open-close'][0]

## Visualize horizontal motion across frames

In [ ]:
sample = data['open-close'][5] # Fifth sample
frame = sample[5][:,:,0] # Middle frame

fig = plt.figure()

im = plt.imshow(frame, animated=True)

def update(i):
    frame = sample[i][:,:,0]
    im.set_array(frame)
    return im,

ani = animation.FuncAnimation(fig, update, frames = range(10),interval=200,repeat=True)
plt.show()